In [34]:
!pip install fredapi

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\rahul\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [35]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\rahul\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [36]:
!nvidia-smi

NVIDIA-SMI has failed because you do not have suffient permissions. Please try running as an administrator.


In [37]:
import pandas as pd
import torch 
import matplotlib.pyplot as plt
from fredapi import Fred
import numpy as np

In [38]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ["API_KEY"]

In [39]:
fred = Fred(api_key=api_key)
data = fred.get_series("NROU")
data.tail()

2033-10-01    4.236570
2034-01-01    4.230920
2034-04-01    4.225190
2034-07-01    4.219381
2034-10-01    4.213503
dtype: float64

In [40]:
data = pd.DataFrame(data,columns=['Rate'])
data.reset_index(inplace=True)
data.head()

,index,Rate
0,1949-01-01,5.255053
1,1949-04-01,5.261516
2,1949-07-01,5.268013
3,1949-10-01,5.274564
4,1950-01-01,5.281182


In [41]:
data.rename(columns={"index": "Date"}, inplace=True)
data.head()

,Date,Rate
0,1949-01-01,5.255053
1,1949-04-01,5.261516
2,1949-07-01,5.268013
3,1949-10-01,5.274564
4,1950-01-01,5.281182


In [52]:
rates = data["Rate"].to_numpy()
rates_tensor = torch.from_numpy(rates)

In [43]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [58]:
cut_batch = int(len(rates) * .6)
end_batch = int(len(rates) * .2)

X_train = rates_tensor.to(device)
X_eval = rates_tensor[cut_batch:(cut_batch+ end_batch)].to(device)
X_test = rates_tensor[(cut_batch+ end_batch):].to(device)

# len(X_train), len(X_eval), len(X_test)
print(X_train)

tensor([5.2551, 5.2615, 5.2680, 5.2746, 5.2812, 5.2879, 5.2946, 5.3014, 5.3081,
        5.3148, 5.3213, 5.3277, 5.3338, 5.3396, 5.3450, 5.3500, 5.3547, 5.3590,
        5.3630, 5.3666, 5.3699, 5.3730, 5.3759, 5.3786, 5.3812, 5.3837, 5.3862,
        5.3886, 5.3910, 5.3934, 5.3959, 5.3985, 5.4012, 5.4041, 5.4072, 5.4106,
        5.4142, 5.4183, 5.4227, 5.4274, 5.4326, 5.4381, 5.4439, 5.4501, 5.4565,
        5.4631, 5.4698, 5.4768, 5.4839, 5.4912, 5.4987, 5.5065, 5.5146, 5.5230,
        5.5320, 5.5413, 5.5512, 5.5616, 5.5724, 5.5837, 5.5955, 5.6077, 5.6202,
        5.6330, 5.6460, 5.6590, 5.6721, 5.6851, 5.6979, 5.7105, 5.7229, 5.7349,
        5.7467, 5.7583, 5.7697, 5.7810, 5.7923, 5.8037, 5.8152, 5.8269, 5.8389,
        5.8512, 5.8639, 5.8769, 5.8903, 5.9041, 5.9183, 5.9329, 5.9478, 5.9630,
        5.9785, 5.9942, 6.0099, 6.0257, 6.0413, 6.0567, 6.0717, 6.0863, 6.1004,
        6.1138, 6.1266, 6.1387, 6.1501, 6.1608, 6.1708, 6.1802, 6.1889, 6.1970,
        6.2045, 6.2114, 6.2176, 6.2231, 

In [ ]:
import torch.nn as nn
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_layer = nn.Linear(in_features=1, out_features=1)

    def forward(self,x: torch.Tensor) -> torch.Tensor:
        return self.linear_layer(x)

In [ ]:
model0 = LinearRegressionModel()

loss_fn = nn.L1Loss()

optimizer = torch.optim.SGD(model0.parameters(), lr=.01)

In [ ]:
def noLossFr():
    model0.train()
    x_pred = model0.forward(X_train)
    loss = loss_fn()
